In [1]:
import pandas as pd

books = pd.read_csv('books_with_categories.csv')

In [2]:
from transformers import pipeline
pipe = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k = None, device="mps")
pipe("I love this!")

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use mps


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.00852868054062128},
  {'label': 'neutral', 'score': 0.005764600355178118},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.001611993182450533},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [3]:
sentences = books['description'][0].split('.')
predictions = pipe(sentences)

In [4]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [5]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296027541160583},
 {'label': 'neutral', 'score': 0.1403856724500656},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.04794240742921829},
 {'label': 'anger', 'score': 0.009156349115073681},
 {'label': 'disgust', 'score': 0.002628474263474345},
 {'label': 'sadness', 'score': 0.0021221607457846403}]

In [7]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = { label: [] for label in emotion_labels }

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = { label: [] for label in emotion_labels }
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x['label'])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]['score'])
    return { label: np.max(scores) for label, scores in per_emotion_scores.items() }


In [8]:
from tqdm import tqdm

for i in tqdm(range(len(books))):
    isbn.append(books['isbn13'][i])
    sentence = books['description'][i].split('.')
    predictions = pipe(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [39:34<00:00,  2.19it/s]     


In [9]:
emotion_df = pd.DataFrame(emotion_scores)
emotion_df["isbn13"] = isbn

In [10]:
emotion_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273591,0.928169,0.932797,0.646215,0.967158,0.729603,9780002005883
1,0.064134,0.273591,0.928169,0.932797,0.646215,0.967158,0.729603,9780002261982
2,0.064134,0.273591,0.928169,0.932797,0.646215,0.967158,0.729603,9780006178736
3,0.064134,0.273591,0.928169,0.932797,0.646215,0.967158,0.729603,9780006280897
4,0.064134,0.273591,0.928169,0.932797,0.646215,0.967158,0.729603,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.064134,0.273591,0.928169,0.932797,0.646215,0.967158,0.729603,9788172235222
5193,0.064134,0.273591,0.928169,0.932797,0.646215,0.967158,0.729603,9788173031014
5194,0.064134,0.273591,0.928169,0.932797,0.646215,0.967158,0.729603,9788179921623
5195,0.064134,0.273591,0.928169,0.932797,0.646215,0.967158,0.729603,9788185300535


In [11]:
books = pd.merge(books, emotion_df, on='isbn13')

In [12]:
books.to_csv('books_with_emotions.csv', index=False)